In [2]:
import os
dataDir = os.listdir('../dataset/Chess/')
print(dataDir)

baseDir = '../dataset/'

trainData = os.path.join(baseDir,'train')
os.mkdir(trainData)

validationData = os.path.join(baseDir,'validation')
os.mkdir(validationData)

#train folders
trainBishopData = os.path.join(trainData,'Bishop')
os.mkdir(trainBishopData)
trainKingData = os.path.join(trainData,'King')
os.mkdir(trainKingData)
trainKnightData = os.path.join(trainData,'Knight')
os.mkdir(trainKnightData)
trainPawnData = os.path.join(trainData,'Pawn')
os.mkdir(trainPawnData)
trainQueenData = os.path.join(trainData,'Queen')
os.mkdir(trainQueenData)
trainRookData = os.path.join(trainData,'Rook')
os.mkdir(trainRookData)


#validation folders
valBishopData = os.path.join(validationData,'Bishop')
os.mkdir(valBishopData)
valKingData = os.path.join(validationData,'King')
os.mkdir(valKingData)
valKnightData = os.path.join(validationData,'Knight')
os.mkdir(valKnightData)
valPawnData = os.path.join(validationData,'Pawn')
os.mkdir(valPawnData)
valQueenData = os.path.join(validationData,'Queen')
os.mkdir(valQueenData)
valRookData = os.path.join(validationData,'Rook')
os.mkdir(valRookData)

['Bishop', 'King', 'Knight', 'Pawn', 'Queen', 'Rook']


In [3]:
import os 
import random
import shutil

splitRatio = 0.85

dataDir = os.listdir('../dataset/Chess')
print(dataDir)

def split_data(src, training, validation, split_size):
    files = []

    for filename in os.listdir(src):
        file = src + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")
    print(len(files))

    training_length = int(len(files) * split_size)
    validation_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))

    training_set = shuffled_set[0:training_length]
    validation_set = shuffled_set[:validation_length]

    for filename in training_set:
        this_file = src + filename
        destination = training + filename
        shutil.copyfile(this_file, destination)

    for filename in validation_set:
        this_file = src + filename
        destination = validation + filename
        shutil.copyfile(this_file, destination)
    
BishopSrc = '../dataset/Chess/Bishop/'
BishopTrain = '../dataset/train/Bishop/'
BishopVal = '../dataset/validation/Bishop/'

KingSrc = '../dataset/Chess/King/'
KingTrain = '../dataset/train/King/'
KingVal = '../dataset/validation/King/'

KnightSrc = '../dataset/Chess/Knight/'
KnightTrain = '../dataset/train/Knight/'
KnightVal = '../dataset/validation/Knight/'

PawnSrc = '../dataset/Chess/Pawn/'
PawnTrain = '../dataset/train/Pawn/'
PawnVal = '../dataset/validation/Pawn/'

QueenSrc = '../dataset/Chess/Queen/'
QueenTrain = '../dataset/train/Queen/'
QueenVal = '../dataset/validation/Queen/'

RookSrc = '../dataset/Chess/Rook/'
RookTrain = '../dataset/train/Rook/'
RookVal = '../dataset/validation/Rook/'

split_data(BishopSrc, BishopTrain, BishopVal, splitRatio)
split_data(KingSrc, KingTrain, KingVal, splitRatio)
split_data(KnightSrc, KnightTrain, KnightVal, splitRatio)
split_data(PawnSrc, PawnTrain, PawnVal, splitRatio)
split_data(QueenSrc, QueenTrain, QueenVal, splitRatio)
split_data(RookSrc, RookTrain, RookVal, splitRatio)


['Bishop', 'King', 'Knight', 'Pawn', 'Queen', 'Rook']
87
76
106
107
78
102


In [4]:
import tensorflow as tf
from tensorflow.python.keras.layers import Conv2D , MaxPooling2D , Flatten, Dense, Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizers import adam_v2 as adam
from tensorflow.python.keras.callbacks import EarlyStopping , ModelCheckpoint
import matplotlib.pyplot as plt
from glob import glob

In [5]:
imgWidth = 256
imgHeight = 256
batchSize = 32
numOfEpochs = 100

TRAINING_DIR = "../dataset/train/"

NumOfClasses = len(glob('../dataset/train/*')) # dont forget the '  /*  '
print (NumOfClasses) # 6 classes

# data augmentation to increase the train data

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0, #normalize between 0 - 1
                                    rotation_range = 30 ,
                                    zoom_range = 0.4 ,
                                    horizontal_flip=True,
                                    shear_range=0.4)


train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size = batchSize,
                                                    class_mode = 'categorical',
                                                    target_size = (imgHeight,imgWidth))


validation_DIR = "../dataset/validation/"
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)

val_generator = val_datagen.flow_from_directory(validation_DIR,
                                                batch_size = batchSize,
                                                class_mode='categorical',
                                                target_size = (imgHeight, imgWidth))




6
Found 464 images belonging to 6 classes.
Found 87 images belonging to 6 classes.


In [6]:
# early stopping

callBack = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')


# if we will find a better model we will save it here :
bestModelFileName = "../models/chess_best_model.h5"

bestModel = ModelCheckpoint(bestModelFileName, monitor='val_accuracy', verbose=1, save_best_only=True)


In [7]:
model = Sequential([ 
    Conv2D(32, (3,3) , activation='relu' , input_shape=(imgHeight, imgWidth, 3) ) ,
    MaxPooling2D(2,2),
    
    Conv2D(64 , (3,3) , activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(64 , (3,3) , activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128 , (3,3) , activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(256 , (3,3) , activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),

    Dense(512 , activation='relu'),
    Dense(512 , activation='relu'),

    Dense(NumOfClasses , activation='softmax') # softmax -> 0 to 1 
])

print (model.summary() )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 128)       7

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator,
                    epochs = 1,
                    verbose=1,
                    validation_data = val_generator,
                    callbacks = [bestModel])


# display the result using pyplot

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc)) # for the max value in the diagram

     41/Unknown - 54s 1s/step - loss: 1.7940 - accuracy: 0.1859

KeyboardInterrupt: 

In [16]:
# display the result using pyplot

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc)) # for the max value in the diagram


# accuracy chart

fig = plt.figure(figsize=(14,7))
plt.plot(epochs, acc , 'r', label="Train accuracy")
plt.plot(epochs, val_acc , 'b', label="Validation accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Train and validation accuracy')
plt.legend(loc='lower right')
plt.show()

#loss chart
fig2 = plt.figure(figsize=(14,7))
plt.plot(epochs, loss , 'r', label="Train loss")
plt.plot(epochs, val_loss , 'b', label="Validation loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and validation Loss')
plt.legend(loc='upper right')
plt.show()

NameError: name 'history' is not defined